#### Import Modules

In [1]:
import warnings
import pandas as pd
import sys, os


from causalnex.structure import StructureModel

sys.path.append(os.path.abspath(os.path.join('../scripts')))
from eda_pipeline import EDAPipeline
from causal_learning_helper import CausalHelper
warnings.filterwarnings("ignore")

/home/n/anaconda3/envs/env_causal/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
eda_pipeline = EDAPipeline()
preprocessed_trip_df = eda_pipeline.read_data(path="../data/preprocessed_trip.csv", version="v1_preprocessed")
preprocessed_trip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534369 entries, 0 to 534368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        534369 non-null  int64  
 1   Trip ID           534369 non-null  int64  
 2   Trip Origin       534369 non-null  object 
 3   Trip Destination  534369 non-null  object 
 4   Trip Start Time   534369 non-null  object 
 5   Trip End Time     534368 non-null  object 
 6   is_weekend        534369 non-null  int64  
 7   is_holiday        534369 non-null  int64  
 8   distance          534369 non-null  float64
 9   speed             534369 non-null  float64
dtypes: float64(2), int64(4), object(4)
memory usage: 40.8+ MB


In [3]:
df_driver = eda_pipeline.read_data(path="../data/driver_locations_during_request.csv", version="v1_driver")
df_driver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id             1557740 non-null  int64  
 1   order_id       1557740 non-null  int64  
 2   driver_id      1557740 non-null  int64  
 3   driver_action  1557740 non-null  object 
 4   lat            1557740 non-null  float64
 5   lng            1557740 non-null  float64
 6   created_at     0 non-null        float64
 7   updated_at     0 non-null        float64
dtypes: float64(4), int64(3), object(1)
memory usage: 95.1+ MB


#### Merge the two datasets

In [4]:
preprocessed_trip_df['driver_distance'] = preprocessed_trip_df['distance'].apply(lambda x: -1)
preprocessed_trip_df['driver_lat'] = preprocessed_trip_df['distance'].apply(lambda x: 0.0)
preprocessed_trip_df['driver_lng'] = preprocessed_trip_df['distance'].apply(lambda x: 0.0)


In [5]:
causalhelper = CausalHelper()

In [6]:
def combine_get_driver_locations(df_trip, df_driver):
    # df_t = df_trip.copy()
    driver_lat_ls = []
    driver_lng_ls = []
    k = 0
    for i, df in df_trip.iterrows():
        trip_id = df['Trip ID']
        drivers = df_driver[df_driver['order_id']==trip_id]
        driver = drivers[drivers['driver_action']=="accepted"]
        if len(driver)>0:
            print(driver['lat'])
            print(driver['lng'])
            driver_lat_ls.append(float(driver['lat']))
            driver_lng_ls.append(float(driver['lng']))
            # df['driver_lat'] = driver['lat']
            # df['driver_lng'] = driver['lng']
        else:
            driver_lat_ls.append(0.0)
            driver_lng_ls.append(0.0)
        # k+=1
        # if k > 1000:
        #     print(k)
        #     k=0

    return driver_lat_ls, driver_lng_ls   

In [7]:
driver_lat,driver_lng = causalhelper.combine_get_driver_locations(preprocessed_trip_df, df_driver)

In [9]:
preprocessed_trip_df['driver_lat'] = driver_lat
preprocessed_trip_df['driver_lng'] = driver_lng

In [11]:
driver_lat_coordinates = preprocessed_trip_df['driver_lat'].tolist()
driver_lng_coordinates = preprocessed_trip_df['driver_lng'].tolist()
ending_coordinates = []
for i in range(len(driver_lat_coordinates)):
    ending_coordinates.append(f'{driver_lat_coordinates[i]},{driver_lng_coordinates[i]}')

In [12]:
start_coordinates = preprocessed_trip_df['Trip Origin'].tolist()

#### Calculate distance based on coordinates of driver

In [13]:
preprocessed_trip_df['driver_distance'] = eda_pipeline.calculate_distances(start_coordinates, ending_coordinates)


KeyboardInterrupt: 

In [37]:
preprocessed_trip_df[preprocessed_trip_df['driver_lng']!=0.0]

,Unnamed: 0,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,is_weekend,is_holiday,distance,speed,driver_distance,driver_lat,driver_lng
